Performs landmark localisation with initialisation (bbox) from the previous frame. 

Fundamental (preceding) tracker, where the first frame is provided and then export bb from each 
frame to be used for the next one. Adapted from run_online_tracking_all_methods.ipynb.

In [ ]:
from __future__ import division
from os.path import isdir, join, isfile, sep
from os import listdir 
from functools import partial
from warnings import warn
import numpy as np
from research_pyutils.path_related_functions import mkdir_p, rm_if_exists
import logging
from glob import glob

# menpo packages imports
import menpo.io as mio
from menpodetect.dlib.conversion import pointgraph_to_rect
from dlib import shape_predictor
from menpo.shape import PointCloud
from menpo.visualize import print_progress
from menpo.landmark import LandmarkGroup

In [ ]:
# print the name of the computer, useful if running on condor
import socket
import time 
print(socket.gethostname())
print(time.strftime("%d/%m/%Y, %H:%M:%S"))

# Define paths and landmarks localisation methods

In [ ]:
path_base = '/vol/atlas/homes/grigoris/misc/2016_ijcv/data/300vw_testset/'
assert(isdir(path_base))

path_pickles = '/vol/atlas/homes/grigoris/misc/2016_ijcv/data/pickles/'
assert(isdir(path_pickles))

In [ ]:
# run for preceding tracker but for all landmark loc methods
landm_loc_pool = ['dlibERT']  # aam
fold_out_pool = ['tracking_preceding_' + i for i in landm_loc_pool]

# Functions to process frame, clip and list of clips respectively

In [ ]:
def detection_to_pointgraph(detection):
    return PointCloud(np.array([(p.y, p.x) for p in detection.parts()]))


def process_frame(p_fr, lms, model):
    """
    Processes one frame. Given a path to load the image, an lms (from the 
    previous frame) and a model, it fits the model and returns the fitting.
    """
    im = mio.import_image(p_fr)
    if im.n_channels == 3:
        im = im.as_greyscale()
        
    if method_landm_loc == 'dlibERT':
        im_pili = np.array(im.as_PILImage())
        det_frame = model(im_pili, pointgraph_to_rect(lms))
        init_pc = detection_to_pointgraph(det_frame)
        ln_n = init_pc
    else:
        ft = model.fit_from_bb(im, lms)
        ln_n = ft.fitted_image.landmarks['final'].copy()
        del ft  # in menpofit <= 0.3.1, necessary to maintain reasonable ram (huge ft)
    return ln_n, im.path.stem

In [ ]:
def preceding_tracker_clip(p_clip, p_ln, p_out, model, start=0):
    """
    Processes a clip for the preceding tracker. 
    Initialises from the first frame the bb and then uses each frame's 
    landmarks to initialise the next frame.
    """
    assert(isdir(p_clip) and isdir(p_out) and isdir(p_ln))
    frames = sorted(listdir(p_clip))[start : ]  # exclude the first start frames
    assert(len(frames) > 1)
    # initialise the tracker (and the bb) with the first (start) frame
    fr = frames[0]

    # load the landmark and ensure it's a bb
    bb_gl = glob(p_ln + fr[:fr.rfind('.')] + '*.pts')
    assert(len(bb_gl) >= 1)
    ln = mio.import_landmark_file(bb_gl[0])
    bb = ln.lms.bounding_box()
    
    min_sz = np.array([10, 10])  # minimum size of bb we 'allow'. 
    for fr in print_progress(frames):  # iterate over all frames and run the tracker
        bb1, name = process_frame(p_clip + fr, bb, model)
        if isinstance(model, shape_predictor):
            bb1 = LandmarkGroup.init_with_all_label(bb1)

        bb = bb1.lms.bounding_box()  # initialisation bb for the next frame
        mio.export_landmark_file(bb1, p_out + name + '.pts', overwrite=True)
        
        # if the bb is smaller than the min_sz, skip the rest of the frames
        bp = bb.points
        if np.any(np.max(bp, 0) - np.min(bp, 0) < min_sz):
            print()
            return
        

In [ ]:
def call_tracker_clips(p_clips, p_in, p_out, model, start=0):
    # calls the tracker for each clip, assuming pipeline format of folders.
    assert(isdir(p_clips) and isdir(p_in))
    clips = sorted(listdir(p_clips))[7:]
    lcl = len(clips)
    
    for cnt, clip in enumerate(clips):
        print('{} out of {} (Clip {})'.format(cnt + 1, lcl, clip))
        preceding_tracker_clip(join(p_clips, clip, ''), join(p_in, clip, ''), 
                               mkdir_p(join(p_out, clip, '')), model, start=start)

# Track all the frames

In [ ]:
cats = listdir(path_base)
model_bb = 'gt_bb'
gt_fold = 'gt1_1_bb'  # used to load the first bb

for f0 in range(len(fold_out_pool)):
    fold_out = fold_out_pool[f0]
    method_landm_loc = landm_loc_pool[f0]
    print('method: {}, tracker: \'preceding\''.format(fold_out))
    
    # load prediction model
    if method_landm_loc == 'dlibERT':
        path_shape_pred = path_pickles + 'modelln_' +  model_bb + '_' + method_landm_loc + '.model'
        assert(isfile(path_shape_pred))
        model = shape_predictor(path_shape_pred)
    else:
        path_pkl = path_pickles + 'modelln_' +  model_bb + '_' + method_landm_loc + '.pkl'
        assert(isfile(path_pkl))
        model = mio.import_pickle(path_pkl)
        
    # for each category in the testset, run landmark loc method
    for cat in cats:
        if not cat[:8] == 'category' or not isdir(path_base + cat):
            warn('Unknown content in path {} (folder: {}).'.format(path_base, cat))
        print(cat)
        # join or create the paths
        p_fr = join(path_base, cat, 'frames', '')
        p_ln_out_0 = mkdir_p(join(path_base, cat, fold_out, ''))
        p_bb_in = mkdir_p(join(path_base, cat, gt_fold, ''))
        assert(isdir(p_fr) and isdir(p_bb_in))
        
        call_tracker_clips(p_fr, p_bb_in, p_ln_out_0, model)

    del model


In [ ]:
import time 
print(time.strftime("%d/%m/%Y, %H:%M:%S"))